In [1]:
import pandas as pd

## Задание 1

Открой файл [distances.ipynb](src/notebooks/distances.ipynb). 
* Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 
* Оставь в датасете только фильмы, которые вышли в "релиз".
* Убери фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведи количество фильмов, оставшихся в выборке 

In [2]:
df_c = pd.read_csv('../../datasets/tmdb_5000_credits.csv')
df_m = pd.read_csv('../../datasets/tmdb_5000_movies.csv')

new_df = pd.merge(df_m, df_c, left_on = 'id', right_on = 'movie_id') #объединение df
new_df.rename(columns={'title_x': 'title'}, inplace=True) #переименов слотбец title
new_df.drop('title_y', axis=1, inplace=True) #удаляем повторную колону названий
new_df = new_df[new_df['status'] == 'Released'] #фильтрация данных по Released
new_df.dropna(subset=['overview', 'genres', 'keywords'], inplace=True) #убираем фильмы с пропускаи в колонках
new_df.shape[0]

4792

## Задание 2

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объедини тексты этих колонок и проведи предобработку:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитай матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

Выведи размер получившейся матрицы
> Параметр `max_features` в `TfidfVectorizer` должен быть равен 10000

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

new_df['recomend'] = new_df['overview']+ ' ' + new_df['keywords'] #объединяем текст колонок
vector=TfidfVectorizer(max_features=10000, stop_words='english') #удаляем все англ стоп-слова
new_df['recomend'] = new_df['recomend'] .fillna('') #заменяем NaN пустой символ
matrix = vector.fit_transform(new_df['recomend'] ) #создаем матрицу
matrix.shape

(4792, 10000)

## Задание 3

Рассчитай [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составь из этой матрицы `pd.DataFrame`. Для дальнейшего удобства, 
колонки и индексы таблицы назови согласно`id` фильма. \
Сохрани получившийся `DataFrame` c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохрани в папку [assets](src/assets) с названием `movies.csv`.

> Получившиеся файлы `distance.csv` и `movies.csv` пушить в GitLab не нужно!

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(matrix) #применение cosine similarity 
distance = pd.DataFrame(cosine_sim, index=new_df['movie_id'], columns = new_df['movie_id']) #колонки и индексы названы согласно id фильма

#Сохранение датафреймов
distance.to_csv('../../src/assets/distance.csv')
new_df.to_csv('../../src/assets/movies.csv')